Using DataHandler Class:

In [93]:
import os
from DataHandler import DataHandler # Import Library
Data = DataHandler('MEM/Bin Ge/ME2C30612A_A3.MEM') # Create DataHandler object and load file

What is in Data? The following can be retrieved from the file (only if MEM file is loaded):

Data.name # Name of the file

Data.x # X axis data

Data.y # Y axis data

Data.prescan_indices # Indices of prescan data

Data.postscan_indices # Indices of postscan data

Data.x_trimmed # X axis data with prescan and postscan data removed

Data.y_trimmed # Y axis data with prescan and postscan data removed

Therefore, printing the x and y trimmed data:

In [94]:
print(Data.x_trimmed) # Print trimmed x data
print(Data.y_trimmed) # Print trimmed y data

# Plotting the data
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data.x_trimmed, y=Data.y_trimmed, mode='lines+markers'))
fig.show() # Show the plot


[11.135 11.113 11.09  11.068 11.045 11.023 11.002 10.98  10.957 10.935
 10.915 10.892 10.87  10.85  10.827 10.805 10.785 10.762 10.74  10.72
 10.698 10.677 10.655 10.635 10.613 10.592 10.57  10.55  10.528 10.507
 10.485 10.465 10.445 10.422 10.403 10.382 10.36  10.34  10.32  10.3
 10.278 10.257 10.238 10.217 10.198 10.175 10.155 10.135 10.115 10.095
 10.075 10.055 10.035 10.015  9.995  9.975  9.955  9.935  9.915  9.895
  9.875  9.855  9.835  9.818  9.797  9.778  9.757  9.738  9.717  9.7
  9.68   9.66   9.64   9.622  9.602  9.582  9.565  9.545  9.525  9.507
  9.488  9.47   9.45   9.432  9.412  9.395  9.375  9.358  9.337  9.32
  9.3    9.283  9.262  9.245  9.225  9.207  9.19   9.17   9.153  9.135
  9.115  9.097  9.08   9.062  9.042  9.025  9.007  8.99   8.972  8.952
  8.935  8.917  8.9    8.882  8.865  8.847  8.83   8.812  8.795  8.778
  8.757  8.74   8.725  8.707  8.69   8.672  8.655  8.637  8.62   8.602
  8.585  8.568  8.55   8.535  8.517  8.5    8.483  8.465  8.45   8.432
  8.415  8.3

From there, you can play around with the data and do whatever you want with it. We can also go through the CMAPSim class as well now:

I am using arbitrary variables here:

In [95]:
from Simulate import CMAPSim

# Define parameters

Cells = 20      # Number of Motor Units
alpha1 = 0.204    # Exponential Scale parameter (Mean motor unit amplitude (+ minumum))
beta1 = 0.05   # Exponential Location parameter (Minimnum motor unit amplitude)
alpha2 = 26.5     # Activation threshold mean
beta2 = 2       # Activation threshold deviation (gaussian)
alpha3 = 0.015  # Relative spread lower bound (uniform dist)
beta3 = 0.018    # Relative spread upper bound (uniform dist)
length = 400    # Length of simulation (nsamples)
noiseoffset = 0.01  # Additive noise mean
noisedev = 0.005    # Additive noise deviation

# Execute simulation
SimData = CMAPSim(Cells, alpha1, beta1, alpha2, beta2, alpha3, beta3, length, noiseoffset, noisedev, method="Stairfit", seed=101) # Create SimulatedData Object and run simulation, with random seed set to 99 for repeatability

[0.88429646 0.16793962 0.24509394 0.26068147 0.1805605  0.12770603
 0.60596682 0.17721801 0.88052653 0.21926327 0.13407228 0.0589925
 1.31700818 0.86224694 0.10205784 0.28780705 0.36606316 0.36717121
 0.29422794 0.1802252 ]


And as a result, you get SimData.x and SimData.y:

In [96]:
# print(SimData.x) # Print x data
# print(SimData.y) # Print y data

from Export import generateDAT
from datetime import datetime
# generateDAT(datetime.now(), SimData.x, SimData.y)     # Export data to MScanFit-compatible .DAT file

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=SimData.x, y=SimData.y, mode='markers'))
fig.show() # Show the plot

Now, with the data and simulations prepped, we can do our analysis. Starting with STEPIX:

Inputs: Y data, and Noise Threshold

Outputs: STEPIX, AMPIX, D50, and Point (refer to STEPIX paper)

No plotting functionality in this one.

In [97]:
from STEPIX import SETPIX
Stepix=SETPIX(Data.y_trimmed,NoiseThreshold=0.02) # Create SETPIX object and run the algorithm with the default threshold

print(Stepix.STEPIX)
print(Stepix.AMPIX)
print(Stepix.D50)
print(Stepix.Point)

71
61.394366197183096
40
R


Moving on to CDIX:

Inputs: X data, Y data, and Mean_LS, and PlotResult

Outputs: G, NDivs, CDIX (Refer to paper, G is Gridsize, S and D not really relevant)

GMM_means, GMM_std_dev, and GMM_weights relevant as well to monitor the Gaussian Mixed Modeling Results

When PlotResult is true, shows a CDIX plot similar to the paper. For Mean_LS, refer to CDIX Paper

In [98]:
from CDIX import CDIX
CDIX=CDIX(Data.x,Data.y,Mean_LS=1,PlotResult=True) # Create CDIX object and run the algorithm with Mean_LS of 1. PlotResult is set to True to show the plot. Also note that we are using untrimmed Data for this algorithm.

print("Final Means:", CDIX.GMM_means)
print("Final Standard Deviations:", CDIX.GMM_std_dev)
print("Final Weights:", CDIX.GMM_weights)

print("Grid Size is: %f" % CDIX.G)
print("Number of Divisions is: %d" % CDIX.NDivs)
print("CDIX is: %f" % CDIX.CDIX)

Final Means: [ 1.89351733e-01 -1.89351733e-01  3.10055323e-18]
Final Standard Deviations: [0.10251167 0.10251167 0.0820147 ]
Final Weights: [0.02866128 0.02866128 0.94267743]
Grid Size is: 0.225451
Number of Divisions is: 19
CDIX is: 15.586204


Finally Moving on to Stairfit:

Inputs: X data, Y data, MURange (range object), termination_threshold, Calc_Thres, and PlotResult

MURange will have the lower and upper bounds of what MU # to search for in stairfit. Calc_Thres is a bool, and should be true if using untrimmed data. 

Outputs: M, Lambdas, and Sizes 

M is MUNE, Lambdas is the absolute magnitued of the steps, and Sizes is the step sizes (refer to stairfit paper)

In [99]:
from Stairfit import Stairfit
# Create Stairfit object and run the algorithm with the default threshold. PlotResult is set to True to show the plot. CalcThresh False since using trimmed. MURange 52,200 means starting at 52 and increasing until reaching 200.

Stairfits = Stairfit(Data.x_trimmed, Data.y_trimmed, termination_threshold=0.015, MURange=range(52,200), Calc_Thres=True, PlotResult=True) 

print(Stairfits.M)
# print(Stairfits.Lambdas)
# print(Stairfits.Sizes)

Optimization terminated successfully.
         Current function value: 4.589305
         Iterations: 2
         Function evaluations: 3217


52
